# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: Soft Prompt Tuning para reduzir tempo de inferência em CPU (criar space no Hugging Face)
* Model: unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit
* Evaluation approach:
* Fine-tuning dataset:

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Load Test Dataset

In [1]:
from datasets import load_dataset

test_dataset = load_dataset("bsvaz/hatexplain-processed", split='test', num_proc=8)
print(test_dataset)

Dataset({
    features: ['text', 'label', 'target'],
    num_rows: 1924
})


In [2]:
import yaml
# Load the YAML file containing the prompt template
with open('prompts.yaml', 'r') as stream:
    prompt_templates = yaml.safe_load(stream)

print(prompt_templates)

{'prompt_inference_template': 'Classify the text below as Hate Speech, Offensive Language, or Normal, use the examples below to help you.\n\n- Labeled examples:\n    - Offensive: <user> jokes on you because i wouldnt respect any professor who tried to teach me queer theory if they didnt look like a slut\n    - Hate Speech: the nigger is hoping the cops beat him up and arrest him so he can wail and cry on camera and get a huge ghetto lottery payout 🐒\n    - Normal: stand up and be counted feel free to message me with any questions the ikk kk is currently taking applications we are looking for honest christian men and women who are ready to make a change ik org\n    - Offensive: yep i am all for more black conservatives being spawned but that not our job low hanging fruit is a bazillian retarded whites\n    - Normal: <user> <user> <user> yes because threatening a girl with rape is a serious crime sharing her info on social media encouraging others to rape her is a serious crime too\n    

In [3]:
from datasets import Dataset

# Convert your data to a HuggingFace dataset
formatted_prompts = [prompt_templates['prompt_inference_template'].format(text) for text in test_dataset['text']]

test_dataset = Dataset.from_dict({
    'text': test_dataset['text'],
    'prompt': formatted_prompts,
    'label': test_dataset['label'],
    'messages': [[{'role': 'user', 'content': formatted_prompt}] for formatted_prompt in formatted_prompts]
})

### Inference

In [4]:
model_name = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'

In [5]:
from transformers import pipeline
from utils import generation_parameters
import torch

# Check GPU availability
device = 0 if torch.cuda.is_available() else -1
if device == 0:
    print("Using GPU")
else:  
    print("Using CPU")

pipe = pipeline('text-generation', model=model_name, device=device, **generation_parameters)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Using GPU


Device set to use cuda:0


In [6]:
import pandas as pd

test_results = pd.DataFrame(columns=['text', 'label', 'prediction'])
test_results.head()

,text,label,prediction


In [7]:
counter_idx = 0

In [8]:
from tqdm.auto import tqdm
from transformers.pipelines.pt_utils import KeyDataset
import time
import gc

batch_size = 32
start_time = time.time()
data_size = test_dataset.num_rows

for i in tqdm(range(counter_idx, len(test_dataset), batch_size)):
    batch = test_dataset['messages'][i:i + batch_size]
    outputs = pipe(batch, batch_size=batch_size)
    
    # Extract the texts and labels from the batch
    batch_texts = test_dataset['text'][i:i + batch_size]
    batch_labels = test_dataset['label'][i:i + batch_size]

    # Process each output and add to test_results
    for j in range(len(outputs)):
        if i + j < data_size:  # Ensure we don't go out of bounds
            response = outputs[j][0]['generated_text']
            
            # Add to test_results DataFrame
            test_results.loc[len(test_results)] = {
                'text': batch_texts[j],
                'label': batch_labels[j],
                'prediction': response
            }
            # Save after each batch to avoid data loss
            test_results.to_csv('test_results.csv', index=True)
    
    counter_idx += batch_size
    torch.cuda.empty_cache()  # Clear GPU memory
    gc.collect()

    # Log progress after each batch
    print(f"Processed batch {i // batch_size + 1}/{(data_size + batch_size - 1) // batch_size}")
    

  0%|          | 0/61 [00:00<?, ?it/s]

Processed batch 1/61
Processed batch 2/61
Processed batch 3/61
Processed batch 4/61
Processed batch 5/61
Processed batch 6/61
Processed batch 7/61
Processed batch 8/61
Processed batch 9/61


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processed batch 10/61
Processed batch 11/61
Processed batch 12/61
Processed batch 13/61
Processed batch 14/61
Processed batch 15/61
Processed batch 16/61
Processed batch 17/61
Processed batch 18/61
Processed batch 19/61
Processed batch 20/61
Processed batch 21/61
Processed batch 22/61
Processed batch 23/61
Processed batch 24/61
Processed batch 25/61
Processed batch 26/61
Processed batch 27/61
Processed batch 28/61
Processed batch 29/61
Processed batch 30/61
Processed batch 31/61
Processed batch 32/61
Processed batch 33/61
Processed batch 34/61
Processed batch 35/61
Processed batch 36/61
Processed batch 37/61
Processed batch 38/61
Processed batch 39/61
Processed batch 40/61
Processed batch 41/61
Processed batch 42/61
Processed batch 43/61
Processed batch 44/61
Processed batch 45/61
Processed batch 46/61
Processed batch 47/61
Processed batch 48/61
Processed batch 49/61
Processed batch 50/61
Processed batch 51/61
Processed batch 52/61
Processed batch 53/61
Processed batch 54/61
Processed 

### Evaluate Inference Results

In [1]:
import pandas as pd

test_results_df = pd.read_csv('test_results.csv')
test_results_df.head()
test_results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1924 entries, 0 to 1923
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        1924 non-null   object
 1   label       1924 non-null   int64 
 2   prediction  1924 non-null   object
dtypes: int64(1), object(2)
memory usage: 45.2+ KB


In [2]:
import ast

# Parse the 'prediction' column to convert strings to lists
test_results_df['prediction'] = test_results_df['prediction'].apply(ast.literal_eval)

# Verify the change
print(test_results_df['prediction'].head())
print(type(test_results_df['prediction'][0]))

0    [{'content': 'Classify the text below as Hate ...
1    [{'content': 'Classify the text below as Hate ...
2    [{'content': 'Classify the text below as Hate ...
3    [{'content': 'Classify the text below as Hate ...
4    [{'content': 'Classify the text below as Hate ...
Name: prediction, dtype: object
<class 'list'>


In [3]:
test_results_df['prediction'][0][1]

{'role': 'assistant',
 'content': 'Alright, so I\'m trying to classify this text into hate speech, offensive language, or normal. Let me read through it carefully.\n\nThe text is: "<user> men can not be raped can not be abuse that why they call it violence against women and children because men are always the perpetrators women come first even before children"\n\nHmm, okay. So the speaker is talking about men being unable to be raped or abused, and they\'re pointing out that violence is against women and children. They say that men are always the perpetrators and that women come first. \n\nI remember the examples given. Offensive language is when someone makes comments that target someone else\'s appearance or background, like looking like a slut. That doesn\'t seem to fit here. \n\nHate speech is when someone attacks another group or person with harsh words, often with a sense of judgment or superiority. The speaker here is not attacking anyone negatively; they\'re pointing out a ster

In [4]:
# Add "<think>" at the beginning of 'content' in the second item of each prediction list
for i in range(len(test_results_df)):
    if len(test_results_df['prediction'][i]) > 1:  # Make sure there's a second item
        if 'content' in test_results_df['prediction'][i][1]:
            test_results_df['prediction'][i][1]['content'] = "<think>" + test_results_df['prediction'][i][1]['content']

In [5]:
from utils import extract_components
extracted = extract_components(test_results_df['prediction'][0][1]['content'])

In [6]:
# Apply extract_components to each row in test_results_df
extracted_data = []
for i in range(len(test_results_df)):
	if len(test_results_df['prediction'][i]) > 1 and 'content' in test_results_df['prediction'][i][1]:
		extracted_data.append(extract_components(test_results_df['prediction'][i][1]['content']))
	else:
		# Handle cases where the prediction doesn't have enough elements or right structure
		extracted_data.append({'full_response': None, 'thinking': None, 'predicted_label': None})

# Create DataFrame from the extracted data
processed_results = pd.DataFrame(extracted_data)
processed_results.head()

,full_response,thinking,predicted_label
0,"<think>Alright, so I'm trying to classify this...","Alright, so I'm trying to classify this text i...",Normal
1,"<think>Okay, so I'm trying to figure out wheth...","Okay, so I'm trying to figure out whether the ...",Normal
2,"<think>Okay, so I have this text: ""<user> <use...","Okay, so I have this text: ""<user> <user> why ...",Normal
3,"<think>Okay, so I'm looking at this text: ""<us...","Okay, so I'm looking at this text: ""<user> <us...",Offensive Language
4,"<think>Alright, let's try to figure out what t...","Alright, let's try to figure out what the user...",Offensive Language


In [7]:
from utils import label_map

references = list(pd.read_csv('test_data.csv')['label'])
print(references[:5])

predictions = processed_results['predicted_label'].tolist()
print(predictions[:5])

[1, 1, 2, 0, 2]
['Normal', 'Normal', 'Normal', 'Offensive Language', 'Offensive Language']


In [8]:
unique_predictions = set(predictions)
print("Unique prediction classes:", unique_predictions)

Unique prediction classes: {'Hate Speech', None, 'Normal', 'Offensive Language'}


In [9]:
import evaluate

# Load evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

# Create reverse label mapping without None
reverse_label_map = {v: k for k, v in label_map.items()}
reverse_label_map_pred = {'Hate Speech': 0, 'Normal': 1, 'Offensive Language': 2}

# Filter out None values from predictions
valid_indices = [i for i, pred in enumerate(predictions) if pred is not None]
valid_predictions = [predictions[i] for i in valid_indices]
valid_references = [references[i] for i in valid_indices]

# Convert valid predictions to integers
valid_predictions_int = [reverse_label_map_pred[pred] for pred in valid_predictions]

# Compute metrics only on valid data
acc_result = accuracy_metric.compute(predictions=valid_predictions_int, references=valid_references)
f1_result = f1_metric.compute(predictions=valid_predictions_int, references=valid_references, average="weighted")

print(f"Computed metrics on {len(valid_predictions)} out of {len(predictions)} examples")
print("Accuracy:", acc_result)
print("Weighted F1 Score:", f1_result)

Computed metrics on 1921 out of 1924 examples
Accuracy: {'accuracy': 0.3909422175950026}
Weighted F1 Score: {'f1': 0.32953267428470956}


In [ ]:
from datasets import load_dataset

dataset = load_dataset('csv', 'test_prompts.csv')
dataset

Generating train split: 0 examples [00:00, ? examples/s]

ValueError: Number of processes must be at least 1

## Preparing the training dataset

In [1]:
!python generate_train_data.py

Loading dataset...
Creating prompts CSV file...
Loaded 43 previously processed results.
Starting phase for label: Offensive Language
Already processed 20 rows for label: Offensive Language
Sending 50 asynchronous requests...
Phase Offensive Language: correct_count = 32
Sending 50 asynchronous requests...
Phase Offensive Language: correct_count = 59
Sending 50 asynchronous requests...
Phase Offensive Language: correct_count = 90
Sending 50 asynchronous requests...
Phase Offensive Language: correct_count = 118
Sending 50 asynchronous requests...
Phase Offensive Language: correct_count = 137
Sending 50 asynchronous requests...
Phase Offensive Language: correct_count = 161
Sending 50 asynchronous requests...
Phase Offensive Language: correct_count = 184
Sending 50 asynchronous requests...
Phase Offensive Language: correct_count = 213
Sending 50 asynchronous requests...
Phase Offensive Language: correct_count = 238
Sending 50 asynchronous requests...
Phase Offensive Language: correct_count 

In [2]:
import pandas as pd

reasoning_df = pd.read_csv('train_reasoning_data.csv')
reasoning_df.rename(columns={'label': 'predicted_label'}, inplace=True)
reasoning_df.head()

,text,predicted_label,thinking,full_responses
0,if english is not imposition then hindi is als...,Offensive Language,"Okay, let me try to work through this classifi...","<think>\nOkay, let me try to work through this..."
1,he said bro even your texts sound redneck what...,Offensive Language,"Okay, let's see. I need to classify the text ""...","<think>\nOkay, let's see. I need to classify t..."
2,i am sure you are totally the alpha faggot at ...,Offensive Language,"Okay, let me try to figure out how to classify...","<think>\nOkay, let me try to figure out how to..."
3,<user> aint he bisexual an lgbt king of gays e...,Offensive Language,"Okay, let's tackle this classification step by...","<think>\nOkay, let's tackle this classificatio..."
4,<user> agree with ananya baby every hindu musl...,Offensive Language,"Okay, let's try to classify this text. The inp...","<think>\nOkay, let's try to classify this text..."


In [3]:
reasoning_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             1500 non-null   object
 1   predicted_label  1500 non-null   object
 2   thinking         1500 non-null   object
 3   full_responses   1500 non-null   object
dtypes: object(4)
memory usage: 47.0+ KB


In [4]:
label_map = {0: 'Hate Speech', 1: 'Normal', 2: 'Offensive Language'}

train_df = pd.read_csv('train_data.csv')
train_df['label'] = train_df['label'].map(label_map)
train_df.head()

,text,label,target
0,u really think i would not have been raped by ...,Offensive Language,"['Hindu', 'Other', 'Islam']"
1,the uk has threatened to return radioactive wa...,Offensive Language,"['Indian', 'Refugee', 'Minority', 'Islam']"
2,if english is not imposition then hindi is als...,Offensive Language,"['Hindu', 'Other']"
3,no liberal congratulated hindu refugees post c...,Offensive Language,['Hindu']
4,he said bro even your texts sound redneck what...,Offensive Language,"['Caucasian', 'Economic']"


In [5]:
# Merge the dataframes using inner join on 'text' column
merged_df = pd.merge(reasoning_df, train_df, on='text', how='left')[['text', 'predicted_label', 'label', 'thinking']]

# Display info about the merged dataframe
print("Merged DataFrame Info:")
print("-" * 20)
merged_df.info()
print("\nFirst few rows:")
print("-" * 20)
merged_df.head()

Merged DataFrame Info:
--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1517 entries, 0 to 1516
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             1517 non-null   object
 1   predicted_label  1517 non-null   object
 2   label            1517 non-null   object
 3   thinking         1517 non-null   object
dtypes: object(4)
memory usage: 47.5+ KB

First few rows:
--------------------


,text,predicted_label,label,thinking
0,if english is not imposition then hindi is als...,Offensive Language,Offensive Language,"Okay, let me try to work through this classifi..."
1,he said bro even your texts sound redneck what...,Offensive Language,Offensive Language,"Okay, let's see. I need to classify the text ""..."
2,i am sure you are totally the alpha faggot at ...,Offensive Language,Offensive Language,"Okay, let me try to figure out how to classify..."
3,<user> aint he bisexual an lgbt king of gays e...,Offensive Language,Offensive Language,"Okay, let's tackle this classification step by..."
4,<user> agree with ananya baby every hindu musl...,Offensive Language,Offensive Language,"Okay, let's try to classify this text. The inp..."


In [7]:
merged_df.drop_duplicates(subset=['text'], inplace=True)

In [8]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1499 entries, 0 to 1516
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             1499 non-null   object
 1   predicted_label  1499 non-null   object
 2   label            1499 non-null   object
 3   thinking         1499 non-null   object
dtypes: object(4)
memory usage: 58.6+ KB


In [9]:
from utils import calculate_response_accuracy

# Calculate response accuracy
calculate_response_accuracy(merged_df)

Accuracy: 99.93%


In [10]:
# Find rows where 'label' and 'predicted_label' are different
different_labels_df = merged_df[merged_df['label'] != merged_df['predicted_label']]

# Print the number of rows where labels are different
print(f"Number of rows where 'label' and 'predicted_label' are different: {len(different_labels_df)}")

# Display the rows where labels are different
different_labels_df.head()

Number of rows where 'label' and 'predicted_label' are different: 1


,text,predicted_label,label,thinking
429,<user> i hate you,Offensive Language,Normal,"Okay, let's try to classify the text ""<user> i..."


In [11]:
merged_df = merged_df.drop(429)

In [12]:
# Calculate response accuracy
calculate_response_accuracy(merged_df)

Accuracy: 100.00%


In [13]:
merged_df.to_csv('train_reasoning_data.csv', index=False)

In [2]:
import pandas as pd

df = pd.read_csv('train_reasoning_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498 entries, 0 to 1497
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             1498 non-null   object
 1   predicted_label  1498 non-null   object
 2   label            1498 non-null   object
 3   thinking         1498 non-null   object
dtypes: object(4)
memory usage: 46.9+ KB


In [3]:
df.head()

,text,predicted_label,label,thinking
0,if english is not imposition then hindi is als...,Offensive Language,Offensive Language,"Okay, let me try to work through this classifi..."
1,he said bro even your texts sound redneck what...,Offensive Language,Offensive Language,"Okay, let's see. I need to classify the text ""..."
2,i am sure you are totally the alpha faggot at ...,Offensive Language,Offensive Language,"Okay, let me try to figure out how to classify..."
3,<user> aint he bisexual an lgbt king of gays e...,Offensive Language,Offensive Language,"Okay, let's tackle this classification step by..."
4,<user> agree with ananya baby every hindu musl...,Offensive Language,Offensive Language,"Okay, let's try to classify this text. The inp..."


In [4]:
import yaml

# Load the YAML file containing the prompt template
with open('prompts.yaml', 'r') as stream:
    prompt_template = yaml.safe_load(stream)['prompt_inference_template']

print(prompt_template)

Classify the text below as Hate Speech, Offensive Language, or Normal, use the examples below to help you.
    
- Labeled examples:
    - Offensive Language: <user> jokes on you because i wouldnt respect any professor who tried to teach me queer theory if they didnt look like a slut
    - Hate Speech: the nigger is hoping the cops beat him up and arrest him so he can wail and cry on camera and get a huge ghetto lottery payout 🐒
    - Normal: stand up and be counted feel free to message me with any questions the ikk kk is currently taking applications we are looking for honest christian men and women who are ready to make a change ik org
    - Offensive Language: yep i am all for more black conservatives being spawned but that not our job low hanging fruit is a bazillian retarded whites
    - Normal: <user> <user> <user> yes because threatening a girl with rape is a serious crime sharing her info on social media encouraging others to rape her is a serious crime too
    - Hate Speech: qu

In [6]:
import json

# Create a list to store the formatted examples
formatted_examples = []

# Process each row in the dataframe
for _, row in df.iterrows():
    # Format the prompt with the text
    formatted_prompt = prompt_template.format(row['text'])
    
    # Format the assistant response
    assistant_response = row['thinking'] + "</think>\n\n" + row['label']
    
    # Create the message object
    message_obj = {
        "messages": [
            {"role": "user", "content": formatted_prompt},
            {"role": "assistant", "content": assistant_response}
        ]
    }
    
    # Add to our examples list
    formatted_examples.append(message_obj)

# Write to JSONL file
with open('training_data.jsonl', 'w', encoding='utf-8') as f:
    for example in formatted_examples:
        f.write(json.dumps(example, ensure_ascii=False) + '\n')

print(f"Successfully created training_data.jsonl with {len(formatted_examples)} examples")

Successfully created training_data.jsonl with 1498 examples


In [10]:
from datasets import load_dataset
from huggingface_hub import login

# Assuming you have a Hugging Face token as an environment variable or will enter it when prompted
login()

# Load the dataset from the JSONL file
dataset = load_dataset('json', data_files='training_data.jsonl')

# Push the dataset to the Hugging Face Hub
# Replace 'your-username/your-dataset-name' with your desired repository name
dataset.push_to_hub('bsvaz/hatexplain-reasoning-dataset', private=False)

print("Dataset successfully pushed to Hugging Face Hub!")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Dataset successfully pushed to Hugging Face Hub!


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.